# Self and Cross Test

In [ ]:
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.time import Time
from astropy.constants import c
from scipy import interpolate
import pickle
from mpfit import mpfit

from tqdm import tqdm
import seaborn as sns

from waveCal import *

## Gather Files

In [ ]:
# LFC
lfc_files = glob('/mnt/home/lzhao/ceph/lfc5*/LFC_*.fits')
ckpt_files = glob('/mnt/home/lzhao/ceph/ckpt5*/LFC_19*.npy')
lfc_files, lfc_times = sortFiles(lfc_files, get_mjd=True)
ckpt_files = sortFiles(ckpt_files)
num_lfc_files = len(lfc_files)
print(num_lfc_files)

hdus = fits.open(lfc_files[0])
t_spec = hdus[1].data['spectrum'].copy()
t_errs = hdus[1].data['uncertainty'].copy()
t_mask = hdus[1].data['pixel_mask'].copy()
hdus.close()
nord, npix = t_spec.shape

lfc_orders = range(45,76)

In [ ]:
# ThAr
thar_files = glob('/mnt/home/lzhao/ceph/thar5*/ThAr_*.fits')
thid_files  = glob('/mnt/home/lzhao/ceph/thid5*/ThAr_*.thid')
thar_files, thar_times = sortFiles(thar_files, get_mjd=True)
thar_files = thar_files[1:] # First file is from before LFCs
thar_times = thar_times[1:]
thid_files = sortFiles(thid_files) [1:]
num_thar_files = len(thar_files)
print(num_thar_files)

## Get and Save Patch Dictionaries

In [ ]:
ckpt_patch = pickle.load(open('./191204_ckptPatch9.pkl','rb'))
thid_patch = pickle.load(open('./191204_thidPatch15.pkl','rb'))

# Self Tests

## LFC

In [ ]:
nfile=6
print(os.path.basename(ckpt_files[nfile]))
print(Time(lfc_times[nfile],format='mjd').isot)

denoised_x = evalWaveSol([lfc_times[nfile]], ckpt_patch, t_intp_deg=3)[0]

### With Denoised X Values
(Should be zero to machine error)

In [ ]:
m = ckpt_patch['orders'].copy()
w = ckpt_patch['waves'].copy()
newx = ckpt_patch['denoised_xs'].copy()[nfile]
newm = m.copy()
w_fit = interp_train_and_predict(newx, newm, denoised_x, m, w, e=None, interp_deg=3)

In [ ]:
good_mask = np.isfinite(w_fit)
resid = w_fit[good_mask]-w[good_mask]
rv_shift = resid/w[good_mask]*c.value

plt.figure()
plt.title('LFC to LFC Denoised xs')
plt.xlabel('Predicted - Fit [A]')
plt.ylabel('Frequency')
#plt.hist(rv_shift,50);
plt.hist(resid,50);
plt.tight_layout()
plt.savefig('./Figures/191204_lfcLfcDenoised.png')

### With Fitted s Values
(Shows error floor of our data-based tests)

In [ ]:
nfile=6
print(os.path.basename(ckpt_files[nfile]))
print(Time(lfc_times[nfile],format='mjd').isot)

denoised_x = evalWaveSol([lfc_times[nfile]], ckpt_patch, t_intp_deg=3)[0]

In [ ]:
newx, newm, neww, newe = readParams(ckpt_files[nfile])
w_fit = interp_train_and_predict(newx, newm, denoised_x, m, w, e=newe, interp_deg=3)

In [ ]:
good_mask = np.isfinite(w_fit)
resid = w_fit[good_mask]-neww[good_mask]
rv_shift = resid/neww[good_mask]*c.value

plt.figure()
plt.title('LFC to LFC Measured xs')
plt.xlabel('Predicted - Fit [m/s]')
plt.ylabel('Frequency')
plt.hist(rv_shift,50);
plt.tight_layout()
plt.savefig('./Figures/191204_lfcLfcMeasured.png')

## ThAr

In [ ]:
nfile=11
print(os.path.basename(thid_files[nfile]))
print(Time(thar_times[nfile],format='mjd').isot)

denoised_x = evalWaveSol(thar_times[nfile], thid_patch, t_intp_deg=3)[0]
m = thid_patch['orders'].copy()
w = thid_patch['waves'].copy()

### With Denoised X Values
(Should be zero to machine error)

In [ ]:
newx = thid_patch['denoised_xs'].copy()[nfile]
newm = m.copy()
w_fit = interp_train_and_predict(newx, newm, denoised_x, m, w, e=None, interp_deg=3)

In [ ]:
good_mask = np.isfinite(w_fit)
resid = w_fit[good_mask]-w[good_mask]
rv_shift = resid/w[good_mask]*c.value

plt.figure()
plt.title('ThAr to ThAr Denoised xs')
plt.xlabel('Predicted - Fit [A]')
plt.ylabel('Frequency')
#plt.hist(rv_shift,50);
plt.hist(resid,50);
plt.tight_layout()
plt.savefig('./Figures/191204_tharTharDenoised.png')

### With Fitted s Values
(Shows error floor of our data-based tests)

In [ ]:
newx, newm, neww = readThid(thid_files[nfile])
w_fit = interp_train_and_predict(newx, newm, denoised_x, m, w, e=None, interp_deg=3)

In [ ]:
good_mask = np.isfinite(w_fit)
resid = w_fit[good_mask]-neww[good_mask]
rv_shift = resid/neww[good_mask]*c.value

plt.figure()
plt.title('ThAr to ThAr Measured xs')
plt.xlabel('Predicted - Fit [m/s]')
plt.ylabel('Frequency')
plt.hist(np.sort(rv_shift)[10:-10],50);
plt.tight_layout()
plt.savefig('./Figures/191204_tharTharMeasured.png')

In [ ]:
np.std(np.sort(rv_shift)[10:-10])

# Cross Tests

## LFC -> ThAr

In [ ]:
time_mask = np.logical_and(thar_times > lfc_times.min(), thar_times < lfc_times.max())
denoised_xs = evalWaveSol(thar_times[time_mask], ckpt_patch, t_intp_deg=3)
m = ckpt_patch['orders'].copy()
w = ckpt_patch['waves'].copy()

In [ ]:
thar_fits = []
thar_shifts = np.array([],dtype=float)
thid_x = []
thid_m = []
thid_w = []
for file_num in tqdm(range(np.sum(time_mask))):
    file_name = thid_files[time_mask][file_num]
    try:
        newx,newm,neww = readThid(file_name)
    except ValueError as err:
        continue
    
    try:
        w_fit = interp_train_and_predict(newx, newm,
                                         denoised_xs[file_num], m, w,
                                         e=None, interp_deg=3)
    except:
        print(file_name)
        continue
    
    thid_x.append(newx)
    thid_m.append(newm)
    thid_w.append(neww)
    thar_fits.append(w_fit)
    good_mask = np.isfinite(w_fit)
    thar_shifts = np.concatenate([thar_shifts,
                                 (w_fit[good_mask]-neww[good_mask])/neww[good_mask]*c.value])

In [ ]:
rv_shift = thar_shifts.flatten()

plt.figure()
plt.title('LFC to ThAr')
plt.xlabel('Predicted - Fit [m/s]')
plt.ylabel('Frequency')
plt.hist(rv_shift,50);
plt.tight_layout()
plt.savefig('./Figures/191204_lfcThar.png')
print(np.std(rv_shift))

## ThAr -> LFC

In [ ]:
time_mask = np.logical_and(lfc_times > thar_times.min(), lfc_times < thar_times.max())
denoised_xs = evalWaveSol(lfc_times[time_mask], thid_patch, t_intp_deg=3)
m = thid_patch['orders'].copy()
w = thid_patch['waves'].copy()

In [ ]:
lfc_fits = []
lfc_shifts = []
ckpt_x = []
ckpt_m = []
ckpt_w = []
for file_num in tqdm(range(np.sum(time_mask))):
    file_name = ckpt_files[time_mask][file_num]
    try:
        newx,newm,neww,newe = readParams(file_name)
    except ValueError as err:
        continue
    try:
        w_fit = interp_train_and_predict(newx, newm, 
                                         denoised_xs[file_num], m, w,
                                         orders=range(46,68),
                                         e=newe, interp_deg=3)
    except:
        continue
    
    ckpt_x.append(newx)
    ckpt_m.append(newm)
    ckpt_w.append(neww)
    lfc_fits.append(w_fit)
    good_mask = np.isfinite(w_fit)
    lfc_shifts = np.concatenate([lfc_shifts,
                                 (w_fit[good_mask]-neww[good_mask])/neww[good_mask]*c.value])

In [ ]:
rv_shift

In [ ]:
rv_shift = lfc_shifts.flatten()

plt.figure()
plt.title('ThAr to LFC')
plt.xlabel('Predicted - Fit [m/s]')
plt.ylabel('Frequency')
plt.hist(rv_shift,50);
plt.tight_layout()
plt.savefig('./Figures/191204_tharLfc.png')